<a href="https://colab.research.google.com/github/velasoka-repo/EVA4/blob/master/Session-4/EVA4_S4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Github Link](https://github.com/velasoka-repo/EVA4/tree/master/Session-4)

# Import Required Packages

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn #neural network
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from tqdm import tqdm #progress bar

# Create Required Data Tranformation Objects

In [0]:
to_tensor = transforms.ToTensor() #convert data into tensor
normalize_data = transforms.Normalize(mean=(0.1307,), std=(0.3081,)) #mean and standard deviation for complete dataset (training + testing)
data_tranformation = transforms.Compose([ 
                              to_tensor,
                              normalize_data
])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create Dataset

In [4]:
train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=data_tranformation)
test_dataset = datasets.MNIST(root="./data", train=False, download=True, transform=data_tranformation)

train_dataset, len(train_dataset), test_dataset, len(test_dataset)

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...



Done!


(Dataset MNIST
     Number of datapoints: 60000
     Root location: ./data
     Split: Train
     StandardTransform
 Transform: Compose(
                ToTensor()
                Normalize(mean=(0.1307,), std=(0.3081,))
            ), 60000, Dataset MNIST
     Number of datapoints: 10000
     Root location: ./data
     Split: Test
     StandardTransform
 Transform: Compose(
                ToTensor()
                Normalize(mean=(0.1307,), std=(0.3081,))
            ), 10000)

# Create DataLoader

In [5]:
gpu_available = torch.cuda.is_available()
print(f"GPU available {gpu_available}")
args = {"num_workers": 1, "pin_memory":True} if gpu_available else {}

data_batch_size = 128
torch.manual_seed(1)
if gpu_available:
  torch.cuda.manual_seed(1)

train_dataloader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=data_batch_size, shuffle=True, **args)
test_dataloader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=data_batch_size, shuffle=True, **args)

train_dataloader, len(train_dataloader), test_dataloader, len(test_dataloader)

GPU available True


(<torch.utils.data.dataloader.DataLoader at 0x7f1d28844128>,
 469,
 79)

# Session-2 Code

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(256, 512, 3)
        self.conv6 = nn.Conv2d(512, 1024, 3)
        self.conv7 = nn.Conv2d(1024, 10, 3)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        # x = F.relu(self.conv7(x))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

# Added Batch Normalization & Dropout

In [0]:
class Net(nn.Module):
    def __init__(self, fm_size):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=fm_size, kernel_size=3, padding=1)
        self.batch_norm1 = nn.BatchNorm2d(num_features=fm_size)
        self.conv2 = nn.Conv2d(in_channels=fm_size, out_channels=fm_size, kernel_size=3, padding=1)
        self.batch_norm2 = nn.BatchNorm2d(num_features=fm_size)
        self.pool = nn.MaxPool2d(2, 2)
        fm_size *=2
        self.conv3 = nn.Conv2d(in_channels=fm_size//2, out_channels=fm_size, kernel_size=3, padding=1)
        self.batch_norm3 = nn.BatchNorm2d(num_features=fm_size)
        self.conv4 = nn.Conv2d(in_channels=fm_size, out_channels=fm_size, kernel_size=3, padding=1)
        self.batch_norm4 = nn.BatchNorm2d(num_features=fm_size)
        # fm_size *=2
        self.conv5 = nn.Conv2d(in_channels=fm_size, out_channels=fm_size, kernel_size=3)
        self.batch_norm5 = nn.BatchNorm2d(num_features=fm_size)
        self.conv6 = nn.Conv2d(in_channels=fm_size, out_channels=int(fm_size*(1.5)), kernel_size=3)
        self.batch_norm6 = nn.BatchNorm2d(num_features=int(fm_size*1.5))

        self.conv7 = nn.Conv2d(in_channels=int(fm_size*1.5), out_channels=10, kernel_size=3)

        self.dropout = nn.Dropout2d(p=0.1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.batch_norm1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv2(x)
        x = self.batch_norm2(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.pool(x)
        

        x = self.conv3(x)
        x = self.batch_norm3(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv4(x)
        x = self.batch_norm4(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.pool(x)
        x = self.dropout(x)

        x = self.conv5(x)
        x = self.batch_norm5(x)
        x = F.relu(x)
        x = self.conv6(x)
        x = self.batch_norm6(x)
        x = F.relu(x)

        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

# Viewing Model Parameters

In [0]:
def show_model_summary(fm_size=8):
  !pip install torchsummary
  from torchsummary import summary
  model = Net(fm_size=fm_size).to(device)
  summary(model, input_size=(1, 28, 28))

# Create Train & Test Functions

In [0]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader, position=0, total=len(train_loader))
    correct = 0
    total = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() #clear the earlier calculated gradient
        output = model(data)
        loss = F.nll_loss(output, target) #Negative Likelihood Loss
        loss.backward() #calculate the gradient loss
        optimizer.step() #update the weight
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        total += len(data)
        pbar.set_description(desc= f'loss={loss.item():.5f} batch_id={batch_idx} Train Accuracy={100.*correct/total:.2f}')

In [0]:
def test(model, device, test_loader):
    model.eval() #equivalent to self.train(False)
    test_loss = 0
    correct = 0
    with torch.no_grad(): #don't calculate the gradient for test run
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Test Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

# Run Model Function

In [0]:
def run_model(fm_size=8, epochs=15):

  show_model_summary(fm_size=fm_size)
  model = Net(fm_size=fm_size).to(device) #to(device) moves the model into GPU
  optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) #Stochastic Gradient Descent

  for epoch in range(1, epochs+1):
      print(f"EPOCH: {epoch}")
      train(model, device, train_dataloader, optimizer, epoch)
      test(model, device, test_dataloader)

# Result

In [25]:
run_model(fm_size=10, epochs=20)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/469 [00:00<?, ?it/s]

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 28, 28]             100
       BatchNorm2d-2           [-1, 10, 28, 28]              20
         Dropout2d-3           [-1, 10, 28, 28]               0
            Conv2d-4           [-1, 10, 28, 28]             910
       BatchNorm2d-5           [-1, 10, 28, 28]              20
         Dropout2d-6           [-1, 10, 28, 28]               0
         MaxPool2d-7           [-1, 10, 14, 14]               0
            Conv2d-8           [-1, 20, 14, 14]           1,820
       BatchNorm2d-9           [-1, 20, 14, 14]              40
        Dropout2d-10           [-1, 20, 14, 14]               0
           Conv2d-11           [-1, 20, 14, 14]           3,620
      BatchNorm2d-12           [-1, 20, 14, 14]              40
        Dropout2d-13           [-1, 20, 14, 14]               0
        MaxPool2d-14             [-1, 2

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0532, Test Accuracy: 9826/10000 (98.26%)

EPOCH: 2


  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0195, Test Accuracy: 9933/10000 (99.33%)

EPOCH: 19


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0197, Test Accuracy: 9936/10000 (99.36%)

EPOCH: 20


loss=0.00066 batch_id=468 Train Accuracy=99.45: 100%|██████████| 469/469 [00:16<00:00, 29.18it/s]



Test set: Average loss: 0.0178, Test Accuracy: 9944/10000 (99.44%)

